#**Install dan import Library**

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!apt install ffmpeg

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-vo7wcl2k
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-vo7wcl2k
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=475f59a0fda970959b50209e1a3f0b1a9d91e227e13c1956ad69e8f3f8d418a6
  Stored in directory: /tmp/pip-ephem-wheel-cache-_74i4fxd/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
!pip install gdown
file_id = "1ztg6pUJWopbchKWC4m2lqLH-0utWnJwI" #drive ke payload.json
!gdown --id {file_id} -O payload.json

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ztg6pUJWopbchKWC4m2lqLH-0utWnJwI
To: /content/payload.json
100% 3.52k/3.52k [00:00<00:00, 12.2MB/s]


In [ ]:
!pip install jiwer evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 51.7 MB/s eta 0:00:00


In [ ]:
import json
import os
import re
import gdown
from pydub import AudioSegment
import whisper
import pandas as pd
import csv
from jiwer import wer, cer, Compose, ToLowerCase, RemovePunctuation, RemoveMultipleSpaces, RemoveWhiteSpace

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [ ]:
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.2 MB/s eta 0:00:00


In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 2.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
from groq import Groq
import time
import pprint

#**Load dan Parsing Data**

##Load dan Ekstrak Payload (.json)

In [ ]:
with open("payload.json") as f:
    data = json.load(f)

# Mengambil list interview dari payload.json
interviews = data["data"]["reviewChecklists"]["interviews"]

# Loop dan tampilkan detail tiap video yang berhasil diambil
for i, item in enumerate(interviews, start=1):
    print(f"Video {i}")
    print("Q:", item["question"])
    print("Link:", item["recordedVideoUrl"])


Video 1
Q: Can you share any specific challenges you faced while working on certification and how you overcame them?
Link: https://drive.google.com/file/d/1EY_uUFWdmjrWYP6H9NEnQa0hCrL_PQN5/view?usp=drive_link
Video 2
Q: Can you describe your experience with transfer learning in TensorFlow? How did it benefit your projects?
Link: https://drive.google.com/file/d/12ZPALf3RXb-OwQ1FT78t6NnFDwfJXyqg/view?usp=drive_link
Video 3
Q: Describe a complex TensorFlow model you have built and the steps you took to ensure its accuracy and efficiency.
Link: https://drive.google.com/file/d/10ysNB7AvBd4986wlNDd0mHf86Hx0Rwy8/view?usp=drive_link
Video 4
Q: Explain how to implement dropout in a TensorFlow model and the effect it has on training.
Link: https://drive.google.com/file/d/1dIU0q0oE_XHoeRBt9sQIpkxmtaioMhwx/view?usp=drive_link
Video 5
Q: Describe the process of building a convolutional neural network (CNN) using TensorFlow for image classification.
Link: https://drive.google.com/file/d/1XCRRbgdVhAR

##Download video

In [ ]:
# Membuat folder videos
os.makedirs("videos", exist_ok=True)

In [ ]:
video_infos = []

for i, item in enumerate(interviews, 1):
    question = item["question"]
    video_url = item["recordedVideoUrl"]

    # Ekstrak file dari google drive link
    match = re.search(r"/d/([a-zA-Z0-9_-]+)", video_url)
    if not match:
        print("Tidak bisa menemukan file id untuk video:", video_url)
        continue

    file_id = match.group(1)
    filename = f"videos/video_{i}.mp4"

    print(f"Downloading video {i}")
    gdown.download(id=file_id, output=filename, quiet=False)

    video_infos.append({
        "index": i,
        "filename": filename,
        "question": question
    })
print("\nSemua video selesai di-download!")


Downloading...
From: https://drive.google.com/uc?id=1EY_uUFWdmjrWYP6H9NEnQa0hCrL_PQN5
To: /content/videos/video_1.mp4
100%|██████████| 18.0M/18.0M [00:00<00:00, 62.3MB/s]


Downloading...
From: https://drive.google.com/uc?id=12ZPALf3RXb-OwQ1FT78t6NnFDwfJXyqg
To: /content/videos/video_2.mp4
100%|██████████| 22.0M/22.0M [00:00<00:00, 46.6MB/s]


Downloading...
From: https://drive.google.com/uc?id=10ysNB7AvBd4986wlNDd0mHf86Hx0Rwy8
To: /content/videos/video_3.mp4
100%|██████████| 22.9M/22.9M [00:00<00:00, 44.6MB/s]


Downloading...
From: https://drive.google.com/uc?id=1dIU0q0oE_XHoeRBt9sQIpkxmtaioMhwx
To: /content/videos/video_4.mp4
100%|██████████| 23.1M/23.1M [00:00<00:00, 56.6MB/s]


Downloading...
From: https://drive.google.com/uc?id=1XCRRbgdVhART7s1dqBhLaF79sjzMwZNr
To: /content/videos/video_5.mp4
100%|██████████| 23.0M/23.0M [00:00<00:00, 24.8MB/s]


Semua video selesai di-download!


##Ekstrak audio

In [ ]:
!pip install noisereduce soundfile

In [ ]:
#Membuat folder untuk audios
os.makedirs("audios", exist_ok=True)

In [ ]:
from pydub.effects import normalize
from pydub import AudioSegment, silence

for info in video_infos:
    video_path = info["filename"]
    audio_path = f"audios/audio_{info['index']}.wav"

    print(f"Extracting audio for {video_path}...")

    # 1. Extract audio dari video
    audio = AudioSegment.from_file(video_path)
    audio = normalize(audio)
    audio = audio.set_channels(1).set_frame_rate(16000)
    audio.export(audio_path, format="wav")

    # 2. Load kembali dan bersihkan gumaman / segments pelan
    clean_audio = AudioSegment.from_wav(audio_path)
    chunks = silence.split_on_silence(
        clean_audio,
        min_silence_len=300,
        silence_thresh=-35  # semakin besar, semakin tidak mentolerir suara pelan
    )

    # 3. Jika ada chunk keras → gabungkan
    if len(chunks) > 0:
        cleaned = sum(chunks)
        cleaned.export(audio_path, format="wav")

    # Simpan path-nya
    info["audio"] = audio_path

print("\nSemua audio .wav selesai dibuat!")


Extracting audio for videos/video_1.mp4...
Extracting audio for videos/video_2.mp4...
Extracting audio for videos/video_3.mp4...
Extracting audio for videos/video_4.mp4...
Extracting audio for videos/video_5.mp4...

Semua audio .wav selesai dibuat!


# **PERBANDINGAN MODEL**

#**Whisper**

In [ ]:
# Menghitung confidence score STT
def whisper_confidence(result):
    segments = result.get("segments", [])
    if not segments:
        return 0.0

    logprobs = []
    for seg in segments:
        lp = seg.get("avg_logprob", None)
        if lp is not None:
            logprobs.append(lp)

    if not logprobs:
        return 0.0

    mean_lp = sum(logprobs) / len(logprobs)

    # Asumsi range logprob -5 (jelek) dan 0 (bagus)
    min_lp, max_lp = -5.0, 0.0
    mean_lp = max(min_lp, min(max_lp, mean_lp))

    # Normalisasi ke 0–1
    conf_0_1 = (mean_lp - min_lp) / (max_lp - min_lp)
    return round(conf_0_1 * 100.0, 2)

In [ ]:
model = whisper.load_model("large-v2")
rows = []
transcripts = {}

for info in video_infos:
    result = model.transcribe(
    info["audio"],
    language="en",
    task="transcribe",
    fp16=False,
    temperature=0.0,
    best_of=7,  # Ambil hasil terbaik dari 7 percobaan
    beam_size=5,
    word_timestamps=True)

    text = result["text"]
    stt_conf = whisper_confidence(result)
    print(f"Question {info['index']}: {info['question']}")
    print(f"Transcript           : {text}")
    print(f"Confidence (0–100)   : {stt_conf}%")
    print("-" * 50)

    transcripts[f"question_{info['index']}"] = {
        "question": info["question"],
        "transcript": text,
        "stt_confidence": round(stt_conf, 2)}

    rows.append({
        "index": info["index"],
        "question": info["question"],
        "audio_file": info["audio"],
        "transcript_auto": text,
         "stt_confidence": stt_conf
    })

100%|█████████████████████████████████████| 2.87G/2.87G [00:38<00:00, 81.1MiB/s]


Question 1: Can you share any specific challenges you faced while working on certification and how you overcame them?
Transcript           :  Can you share any specific challenges faced while working on Skyshine and how you overcome them? Ah, okay. Actually, for the challenges, there are some challenges when I took the certifications, especially for the projects I mentioned that I already working with it. The first one is actually to meet the specific accuracy or validation loss for the evaluation matrix. And yeah, actually, that's just need to take some trial and error with different architecture. For example, like we can try to add more layer, more neurons, changes the neurons. Or even I also apply the dropout layer. So yeah, it really helps with the validation loss to become more lower, right? And yeah, I think that's one of the biggest challenges that I have while working on this certification.
Confidence (0–100)   : 91.19%
--------------------------------------------------
Questio

In [ ]:
# Untuk Import ke transripts.json

with open("transcripts.json", "w") as f:
    json.dump(transcripts, f, indent=2)

print("Semua transkrip berhasil disimpan ke transcripts.json")

Semua transkrip berhasil disimpan ke transcripts.json


In [ ]:
df = pd.DataFrame(rows)
df.head()

,index,question,audio_file,transcript_auto,stt_confidence
0,1,Can you share any specific challenges you face...,audios/audio_1.wav,Can you share any specific challenges faced w...,91.19
1,2,Can you describe your experience with transfer...,audios/audio_2.wav,Can you describe your experience with transfe...,92.71
2,3,Describe a complex TensorFlow model you have b...,audios/audio_3.wav,Describe a core model you've built and the st...,91.45
3,4,Explain how to implement dropout in a TensorFl...,audios/audio_4.wav,Explain how to implement dropout in test serv...,90.15
4,5,Describe the process of building a convolution...,audios/audio_5.wav,Describe the process of building a new sectio...,91.75


In [ ]:
df.to_csv("dataset_transcript_whisper.csv", index=False)
print("dataset_transcript_whisper.csv berhasil dibuat!")

dataset_transcript_whisper.csv berhasil dibuat!


##Merging CSV

In [ ]:
data = [
    ["index","question","audio_file","transcript_manual"],
    [1,"Can you share any specific challenges you faced while working on certification and how you overcame them?","audios/audio_1.wav"," Can share any specific challenges you faced while working on certification and how you overcome them ah, ok, actually, uh for the challenges, uh there are some challenges uh when uh i took the certification especially for the the projects i mentioned that uh that i uh i already working uh with it uh the first one is actually uh to to meet to meet yeah to meet the specific accuracy or uh validation loss right for the evaluation matrix and yeah actually that's just need to to take some trial and error uh with with different architecture for example like uh we can uh try to add more uh layer, more neurons, changes the neurons or even uh i also apply the dropout layer so yeah it really helps with the the validation loss to become more lower right and yeah i think that's one of the biggest eh challenges that that i have uh while uh working on this certifications."],
    [2,"Can you describe your experience with transfer learning in TensorFlow? How did it benefit your projects?","audios/audio_2.wav"," Can describe your experience with transfer learning in tensorflow? How do you benefit from the projects? ah, ok, uh About transfer learning is actually uh we use existing train model from tensorflow for example uh like VGG16, VGG19 right uh especially uh for for some cases uh that we need to use deep learning using Keras applications uh for example like image classification, we can use transfer learning uh models which is that's already uh trained model with uh exceptionally uh high accuracy high performance uh yeah even though it's trained with different uh datasets but it it really helps uh to improve uh our uh model performance uh, model accuracy, model loss uh and yeah uh for example like mobile net, VGG19, VGG16 ya, efficient net uh. It will help to improve our our models uh comparing to the one if you use a traditional uh CNN model yeah CNN model uh with the convolutional uh 2d yeah max pooling and yeah it's it's quite good actually to use uh transfer learning it really helps uh with our model performance to improve our model performance."],
    [3,"Describe a complex TensorFlow model you have built and the steps you took to ensure its accuracy and efficiency.","audios/audio_3.wav"," Describe a complex tensorflow model you've built and the steps you took to ensure its accuracy and efficiency hm, uh complex tensorflow model you've built and steps you took to ensure its accuracy hm, ok, uh i will take one of my uh previous project that uh i use, yeah i also use keras tensorflow model uh its it is uh about uh celiac disease prediction ya uh yeah, this is also i use uh the research project for my uh undergraduate thesis yeah for my skripsi and uh i use this model, it's quite challenging uh even though it's uh achieved high accuracy yeah with with some dense layer yeah, with some uh dropout layer and trial and error with the uh also with the uh callback function yeah uh with the neurons but the problem is the dataset is not uh balance yeah so it has the imbalanced uh class datasets yeah and uh the the approach that i use is to yeah just to use uh the technique called smote and yeah synthetic oversampling technique with edited nearest neighbor. yeah which is, uh Basically it's just oversampling and undersampling uh the datasets. uh It helps with the accuracy."],
    [4,"Explain how to implement dropout in a TensorFlow model and the effect it has on training.","audios/audio_4.wav"," Explain how to implement dropout in ah ok dropout in tensorflow model and it test on training. hm Previously, uh I also have uh implement the dropout layer yeah also in the project submission uh within this certifications. and We can just add the dropout layer. uh For example, uh if I'm not mistaken, I have used this dropout layer in the the one that the case is hm yeah image classifications yeah. German traffic, something if I'm not wrong. I have used this dropout layer uh in in in the not in the last uh in the middle in the middle of the layer. uh So there there's a flattened layer right not flattened, the convolutional layer and the flattened layer. uh and I use that dropout layer, uh uh which is uh I used with the rate of 0.2 or 0.5 if if I'm not wrong. uh And then the dense layer and the the last, the output layer right. uh The effect is it will really helps to hm improve our accuracy and lower our validation loss uh by by turning off some of the previous layer. yeah For example like, we have dense layer 64 and the next layer, uh we implement the dropout uh layer with the rate of uh o point five and it will uh turn off randomly each epoch uh for of the previous dense layer uh"],
    [5,"Describe the process of building a convolutional neural network (CNN) using TensorFlow for image classification.","audios/audio_5.wav"," describe the process of building image, okay uh the cnn one, right so, uh at the first time uh of course uh, we need to make sure uh there are split uh the the image folder split for for uh each uh class okay, uh and then we can use uh Keras preprocessing if I'm not mistaken uh image dataset from directory to split the training and the validation uh dataset right, uh yeah of course we can use another another set which is the test dataset, yeah uh but yeah, okay the next one uh we can, uh yeah we can just uh maybe we we need to implement also the image augmentation yeah, data uh data image data augmentation to uh to make our dataset more uh variative, right for example like we can rotate we can zoom it, we can uh crop it yeah, uh and yeah the last thing, yeah of course we we can build our gen model with the convolutional 2d uh specify the filters, the kernel size, the LRF division of course the input the input shape for the first layer and then we can apply the max pooling 2d uh yeah, and the next layer we can just use convolutional 2D, max pooling and uh whatever it is uh and after that uh we apply the flatten layer uh and dropout layer if you want and the last thing uh don't forget to use the dense layer, right for the output, like uh the last."]
    ]

with open("ground_truth.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(data)

print("CSV berhasil dibuat!")

CSV berhasil dibuat!


In [ ]:
df_manual = pd.read_csv("/content/ground_truth.csv")
df_manual.head()

,index,question,audio_file,transcript_manual
0,1,Can you share any specific challenges you face...,audios/audio_1.wav,Can share any specific challenges you faced w...
1,2,Can you describe your experience with transfer...,audios/audio_2.wav,Can describe your experience with transfer le...
2,3,Describe a complex TensorFlow model you have b...,audios/audio_3.wav,Describe a complex tensorflow model you've bu...
3,4,Explain how to implement dropout in a TensorFl...,audios/audio_4.wav,Explain how to implement dropout in ah ok dro...
4,5,Describe the process of building a convolution...,audios/audio_5.wav,"describe the process of building image, okay ..."


In [ ]:
# Normalisasi + hapus filler words
transform = Compose([
    ToLowerCase(),
    RemovePunctuation(),
    RemoveMultipleSpaces(),
    RemoveWhiteSpace(replace_by_space=True),
])

df_manual["transcript_manual_cleaned"] = df_manual["transcript_manual"].apply(transform)

filler_words = r"\b(uh|hmm|oke|ya|yeah|okay|huh|emm|hm|ah|ok)\b"
df_manual["transcript_manual_cleaned"] = df_manual["transcript_manual_cleaned"].apply(
    lambda text: re.sub(filler_words, '', text, flags=re.IGNORECASE))

df_manual.to_csv("ground_truth_cleaned.csv", index=False)
print("ground_truth_cleaned.csv berhasil dibuat")

ground_truth_cleaned.csv berhasil dibuat


In [ ]:
df_manual = pd.read_csv("/content/ground_truth_cleaned.csv")
df_manual.head()

,index,question,audio_file,transcript_manual,transcript_manual_cleaned
0,1,Can you share any specific challenges you face...,audios/audio_1.wav,Can share any specific challenges you faced w...,can share any specific challenges you faced w...
1,2,Can you describe your experience with transfer...,audios/audio_2.wav,Can describe your experience with transfer le...,can describe your experience with transfer le...
2,3,Describe a complex TensorFlow model you have b...,audios/audio_3.wav,Describe a complex tensorflow model you've bu...,describe a complex tensorflow model youve bui...
3,4,Explain how to implement dropout in a TensorFl...,audios/audio_4.wav,Explain how to implement dropout in ah ok dro...,explain how to implement dropout in dropout...
4,5,Describe the process of building a convolution...,audios/audio_5.wav,"describe the process of building image, okay ...",describe the process of building image the ...


In [ ]:
df_auto = pd.read_csv("/content/dataset_transcript_whisper.csv")
df_auto.head()

,index,question,audio_file,transcript_auto,stt_confidence
0,1,Can you share any specific challenges you face...,audios/audio_1.wav,Can you share any specific challenges faced w...,91.19
1,2,Can you describe your experience with transfer...,audios/audio_2.wav,Can you describe your experience with transfe...,92.71
2,3,Describe a complex TensorFlow model you have b...,audios/audio_3.wav,Describe a core model you've built and the st...,91.45
3,4,Explain how to implement dropout in a TensorFl...,audios/audio_4.wav,Explain how to implement dropout in test serv...,90.15
4,5,Describe the process of building a convolution...,audios/audio_5.wav,Describe the process of building a new sectio...,91.75


In [ ]:
df_merge = df_auto.merge(df_manual, on="index", how="inner")

print("Dataset gabungan:")
print(df_merge.columns)
df_merge.head()

Dataset gabungan:
Index(['index', 'question_x', 'audio_file_x', 'transcript_auto',
       'stt_confidence', 'question_y', 'audio_file_y', 'transcript_manual',
       'transcript_manual_cleaned'],
      dtype='object')


,index,question_x,audio_file_x,transcript_auto,stt_confidence,question_y,audio_file_y,transcript_manual,transcript_manual_cleaned
0,1,Can you share any specific challenges you face...,audios/audio_1.wav,Can you share any specific challenges faced w...,91.19,Can you share any specific challenges you face...,audios/audio_1.wav,Can share any specific challenges you faced w...,can share any specific challenges you faced w...
1,2,Can you describe your experience with transfer...,audios/audio_2.wav,Can you describe your experience with transfe...,92.71,Can you describe your experience with transfer...,audios/audio_2.wav,Can describe your experience with transfer le...,can describe your experience with transfer le...
2,3,Describe a complex TensorFlow model you have b...,audios/audio_3.wav,Describe a core model you've built and the st...,91.45,Describe a complex TensorFlow model you have b...,audios/audio_3.wav,Describe a complex tensorflow model you've bu...,describe a complex tensorflow model youve bui...
3,4,Explain how to implement dropout in a TensorFl...,audios/audio_4.wav,Explain how to implement dropout in test serv...,90.15,Explain how to implement dropout in a TensorFl...,audios/audio_4.wav,Explain how to implement dropout in ah ok dro...,explain how to implement dropout in dropout...
4,5,Describe the process of building a convolution...,audios/audio_5.wav,Describe the process of building a new sectio...,91.75,Describe the process of building a convolution...,audios/audio_5.wav,"describe the process of building image, okay ...",describe the process of building image the ...


In [ ]:
df_merge.to_csv("dataset_final_for_training.csv", index=False)
print("Berhasil membuat dataset_final_for_training.csv")

Berhasil membuat dataset_final_for_training.csv


##WER

In [ ]:
df = pd.read_csv("dataset_final_for_training.csv")

wers = []
cers = []

for i, row in df.iterrows():
    ref = row["transcript_manual"]   # ground truth
    hyp = row["transcript_auto"]     # hasil transkrip menggunakan whisper

    wers.append(wer(ref, hyp))
    cers.append(cer(ref, hyp))

df["WER"] = wers
df["CER"] = cers

df.to_csv("dataset_with_metrics.csv", index=False)

print(df[["index","WER","CER"]])
print("\nEvaluasi selesai, hasil disimpan ke dataset_with_metrics.csv")

   index       WER       CER
0      1  0.401235  0.188506
1      2  0.435484  0.162264
2      3  0.492537  0.248624
3      4  0.348936  0.176814
4      5  0.438735  0.200000

Evaluasi selesai, hasil disimpan ke dataset_with_metrics.csv


In [ ]:
print("Rata-rata WER:", df["WER"].mean())
print("Rata-rata CER:", df["CER"].mean())

Rata-rata WER: 0.4233854200760382
Rata-rata CER: 0.19524155259144785


In [ ]:
df.sort_values("WER", ascending=False).head()

,index,question_x,audio_file_x,transcript_auto,stt_confidence,question_y,audio_file_y,transcript_manual,transcript_manual_cleaned,WER,CER
2,3,Describe a complex TensorFlow model you have b...,audios/audio_3.wav,Describe a core model you've built and the st...,91.45,Describe a complex TensorFlow model you have b...,audios/audio_3.wav,Describe a complex tensorflow model you've bu...,describe a complex tensorflow model youve bui...,0.492537,0.248624
4,5,Describe the process of building a convolution...,audios/audio_5.wav,Describe the process of building a new sectio...,91.75,Describe the process of building a convolution...,audios/audio_5.wav,"describe the process of building image, okay ...",describe the process of building image the ...,0.438735,0.200000
1,2,Can you describe your experience with transfer...,audios/audio_2.wav,Can you describe your experience with transfe...,92.71,Can you describe your experience with transfer...,audios/audio_2.wav,Can describe your experience with transfer le...,can describe your experience with transfer le...,0.435484,0.162264
0,1,Can you share any specific challenges you face...,audios/audio_1.wav,Can you share any specific challenges faced w...,91.19,Can you share any specific challenges you face...,audios/audio_1.wav,Can share any specific challenges you faced w...,can share any specific challenges you faced w...,0.401235,0.188506
3,4,Explain how to implement dropout in a TensorFl...,audios/audio_4.wav,Explain how to implement dropout in test serv...,90.15,Explain how to implement dropout in a TensorFl...,audios/audio_4.wav,Explain how to implement dropout in ah ok dro...,explain how to implement dropout in dropout...,0.348936,0.176814


## WER/CER Final Evaluation (Normalisasi + Hapus Filler + Mapping Pengucapan)


In [ ]:
# Load data
df = pd.read_csv("dataset_final_for_training.csv")

# Fungsi preprocessing lengkap (text mapping)
def preprocess_text(text):
    text = text.lower()
    replacements = {
        "vgc16": "vgg16",
        "vgc19": "vgg19",
        "mobilenet": "mobile net",
        "drawout": "dropout",
        "draw out": "dropout",
        "smooth": "smote",
        "core model": "complex tensorflow model",
        "chain model": "cnn model",
        "dash layer": "dense layer",
        "dense c4": "dense layer 64",
        "flat layer": "flatten layer",
        "flat ten": "flatten",
        "per processing": "preprocessing",
        "pre processing": "preprocessing",
        "the set": "dataset",
        "data set": "dataset",
        "set aside": "dataset",
        "the data": "dataset",
        "conversion to be": "convolutional 2d",
        "convolution to d": "convolutional 2d",
        "next pulling": "max pooling",
        "max pooling to d": "max pooling 2d",
        "celia": "celiac",
        "silic": "celiac",
        "celiax": "celiac",
        "sealick": "celiac",
        "scripcy": "skripsi",

        # Tambahan untuk video 3
        "callback": "callback function",
        "scripts": "skripsi",
        "script": "skripsi",
        "transfer model": "tensorflow model",
        "imbalanced class dataset": "imbalanced dataset",
        "core model": "complex tensorflow model",
        "drawout": "dropout",
        "smooth": "smote",
        "transfer model": "tensorflow model",
        "also used": "also use",
        "used": "use",
        "achieved": "achieve",
        "trial and error also with": "trial and error with",
        "just to use the technique called": "just to use the technique",
        "okay": "",
        "accuracy and efficiency complex tensorflow model you ve built": "accuracy and efficiency",
        "ensure accuracy and efficiency complex tensorflow model": "ensure accuracy and efficiency",
        "tensorflow model you ve built to ensure accuracy": "tensorflow model to ensure accuracy",
        "i used": "i use",
        "i also used": "i also use",
        "trial and error also with": "trial and error with",
        "just to use the technique called": "just to use the technique",
        "okay": "",
        "datasetset": "dataset",
        "datasets": "dataset",
        "its it is": "it is",
        "with the also with the": "with the",
        "projects that i used": "projects that i use",
        "the steps you took to ensure": "to ensure",
        "its accuracy and efficiency": "accuracy and efficiency",

        # Tambahan untuk video 5
        "dash": "dense",
        "dash layer": "dense layer",
        "convolutional to d": "convolutional 2d",
        "max pooling to d": "max pooling 2d",
        "flatten layer": "flatten",
        "keras preprocessing": "image data preprocessing",
        "image of condition": "image augmentation",
        "veritative": "variative",
        "l i m definition": "activation function",
        "new section": "image classification",
        "dash": "dense",
        "dash layer": "dense layer",
        "convolutional2d": "convolutional 2d",
        "maxpooling": "max pooling",
        "veritative": "variative",
        "l i m definition": "activation function"}

    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)

    fillers = [
        r"\buh\b", r"\bum\b", r"\bah\b", r"\boh\b", r"\beh\b",
        r"\byeah\b", r"\bokay\b", r"\bok\b", r"\bhm\b", r"\bem\b"]

    for f in fillers:
        text = re.sub(f, " ", text)

    text = re.sub(r"\b(\w+)( \1\b)+", r"\1", text)
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [ ]:
# Terapkan ke GT dan hypothesis
df["ref_no_filler"] = df["transcript_manual"].apply(preprocess_text)
df["hyp_no_filler"] = df["transcript_auto"].apply(preprocess_text)

In [ ]:
# Hitung metrik akhir
df["WER_final"] = df.apply(lambda row: wer(row["ref_no_filler"], row["hyp_no_filler"]), axis=1)
df["CER_final"] = df.apply(lambda row: cer(row["ref_no_filler"], row["hyp_no_filler"]), axis=1)

In [ ]:
global_wer = wer(" ".join(df.ref_no_filler), " ".join(df.hyp_no_filler))
global_cer = cer(" ".join(df.ref_no_filler), " ".join(df.hyp_no_filler))

In [ ]:
# Output Evaluasi per Audio
print("\nHasil per-audio:")
print(df[["index", "WER_final", "CER_final"]].rename(columns={"WER_final": "WER", "CER_final": "CER"}))


Hasil per-audio:
   index       WER       CER
0      1  0.089552  0.067194
1      2  0.084416  0.039957
2      3  0.113924  0.079153
3      4  0.083333  0.063628
4      5  0.091346  0.082166


In [ ]:
print("Global WER:", global_wer)
print("Global CER:", global_cer)
print("Akurasi STT (WER):", 1 - global_wer)
print("Akurasi STT (CER):", 1 - global_cer)

Global WER: 0.09207459207459208
Global CER: 0.06666666666666667
Akurasi STT (WER): 0.9079254079254079
Akurasi STT (CER): 0.9333333333333333


In [ ]:
# Simpan hasil akhir
df.to_csv("final_transcript_metrics.csv", index=False)

#**WAV2VEC2**

In [ ]:
print("Loading Wav2Vec2 model...")
model_name = "facebook/wav2vec2-large-robust-ft-libri-960h"

processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print(f"Model loaded on {device}")

Loading Wav2Vec2 model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Model loaded on cuda


In [ ]:
# Load ground truth
df_gt = pd.read_csv("ground_truth_cleaned.csv")

In [ ]:
# Transkripsi semua audio
transcripts = []
for idx, row in df_gt.iterrows():
    audio_path = row['audio_file']
    print(f"Transcribing: {audio_path}")
    try:
        audio, sr = torchaudio.load(audio_path)
        if sr != 16000:
            audio = torchaudio.transforms.Resample(sr, 16000)(audio)
        if audio.shape[0] > 1:
            audio = torch.mean(audio, dim=0, keepdim=True)
        input_values = processor(audio.squeeze().numpy(), return_tensors="pt", sampling_rate=16000).input_values
        input_values = input_values.to(device)
        with torch.no_grad():
            logits = model(input_values).logits
        pred_ids = torch.argmax(logits, dim=-1)
        text = processor.batch_decode(pred_ids)[0]
    except Exception as e:
        print("Error:", e)
        text = ""
    transcripts.append(text)

df_gt['transcript_wav2vec2'] = transcripts
df_gt.to_csv("dataset_transcript_wav2vec2.csv", index=False)
print("dataset_transcript_wav2vec2.csv dibuat!")

Transcribing: audios/audio_1.wav
Transcribing: audios/audio_2.wav
Transcribing: audios/audio_3.wav
Transcribing: audios/audio_4.wav
Transcribing: audios/audio_5.wav
dataset_transcript_wav2vec2.csv dibuat!


In [ ]:
# Menampilkan hasil transcript wav2vec2
display(df_gt[['index', 'question', 'audio_file', 'transcript_wav2vec2']].head())

,index,question,audio_file,transcript_wav2vec2
0,1,Can you share any specific challenges you face...,audios/audio_1.wav,CASAISPECFECTORIOS FACE WER WOKING ONSTION O L...
1,2,Can you describe your experience with transfer...,audios/audio_2.wav,CAN DESCRIBE YOUR SPIRITS WE TRANSFOR ME N TAN...
2,3,Describe a complex TensorFlow model you have b...,audios/audio_3.wav,ESCRIPOFOR MODEL YOU FIL AN DESTEPS UTOENSION ...
3,4,Explain how to implement dropout in a TensorFl...,audios/audio_4.wav,WI HOL YOU TO EPEEN DROPOTIN A OE DROW OUT TEI...
4,5,Describe the process of building a convolution...,audios/audio_5.wav,RESCRIBE THE POCESITINFICTION OA N SE AN ANON ...


## Merging CSV

In [ ]:
df_gt_full = pd.read_csv("ground_truth.csv")
df_w2v2 = pd.read_csv("dataset_transcript_wav2vec2.csv")

merged = df_gt_full.merge(df_w2v2[['index', 'transcript_wav2vec2']], on="index")
merged.to_csv("dataset_final_for_training_w2v2.csv", index=False)
print("dataset_final_for_training_w2v2.csv dibuat!")

dataset_final_for_training_w2v2.csv dibuat!


## WER

In [ ]:
df = pd.read_csv("/content/dataset_final_for_training_w2v2.csv")

wers = []
cers = []

for i, row in df.iterrows():
    ref = row["transcript_manual"]   # ground truth
    hyp = row["transcript_wav2vec2"] # hasil transkrip menggunakan whisper

    wers.append(wer(ref, hyp))
    cers.append(cer(ref, hyp))

df["WER"] = wers
df["CER"] = cers

df.to_csv("dataset_with_metrics_w2v2.csv", index=False)

print(df[["index","WER","CER"]])
print("\nEvaluasi selesai, hasil disimpan ke dataset_with_metrics_w2v2.csv")

   index       WER       CER
0      1  1.000000  0.856322
1      2  1.000000  0.868868
2      3  1.000000  0.849541
3      4  0.987234  0.823186
4      5  1.000000  0.832411

Evaluasi selesai, hasil disimpan ke dataset_with_metrics_w2v2.csv


In [ ]:
print("Rata-rata WER:", df["WER"].mean())
print("Rata-rata CER:", df["CER"].mean())

Rata-rata WER: 0.9974468085106384
Rata-rata CER: 0.846065620705942


## WER/CER Final Evaluation (Normalisasi + Hapus Filler + Mapping Pengucapan)

In [ ]:
# Preprocessing
def preprocess_text(text):
    text = text.lower()
    replacements = {
        "vgc16": "vgg16", "vgc19": "vgg19", "mobilenet": "mobile net", "drawout": "dropout",
        "draw out": "dropout", "smooth": "smote", "core model": "complex tensorflow model",
        "dash layer": "dense layer", "dense c4": "dense layer 64", "flat ten": "flatten",
        "per processing": "preprocessing", "pre processing": "preprocessing", "data set": "dataset",
        "the set": "dataset", "set aside": "dataset", "the data": "dataset",
        "conversion to be": "convolutional 2d", "convolution to d": "convolutional 2d",
        "next pulling": "max pooling", "max pooling to d": "max pooling 2d",
        "celia": "celiac", "silic": "celiac", "celiax": "celiac", "sealick": "celiac",
        "scripcy": "skripsi", "scripts": "skripsi", "script": "skripsi",
        "callback": "callback function", "transfer model": "tensorflow model",
        "imbalanced class dataset": "imbalanced dataset", "also used": "also use", "used": "use",
        "achieved": "achieve", "trial and error also with": "trial and error with",
        "just to use the technique called": "just to use the technique",
        "accuracy and efficiency complex tensorflow model you ve built": "accuracy and efficiency",
        "ensure accuracy and efficiency complex tensorflow model": "ensure accuracy and efficiency",
        "tensorflow model you ve built to ensure accuracy": "tensorflow model to ensure accuracy",
        "i used": "i use", "i also used": "i also use", "okay": "",
        "datasetset": "dataset", "datasets": "dataset", "its it is": "it is",
        "with the also with the": "with the", "projects that i used": "projects that i use",
        "the steps you took to ensure": "to ensure", "its accuracy and efficiency": "accuracy and efficiency",
        "dash": "dense", "convolutional to d": "convolutional 2d",
        "keras preprocessing": "image data preprocessing",
        "image of condition": "image augmentation",
        "veritative": "variative", "l i m definition": "activation function",
        "new section": "image classification", "convolutional2d": "convolutional 2d",
        "maxpooling": "max pooling"
    }

    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)

    # Hapus filler words umum
    fillers = [
        r"\buh\b", r"\bum\b", r"\bah\b", r"\boh\b", r"\beh\b",
        r"\byeah\b", r"\bokay\b", r"\bok\b", r"\bhm\b", r"\bem\b"
    ]
    for f in fillers:
        text = re.sub(f, " ", text)

    # Hapus duplikasi kata, simbol, dan normalisasi whitespace
    text = re.sub(r"\b(\w+)( \1\b)+", r"\1", text)  # Menghapus pengulangan kata
    text = re.sub(r"[^a-z0-9\s]", " ", text)        # Menghapus alphanumeric
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [ ]:
# Terapkan
df = pd.read_csv("dataset_final_for_training_w2v2.csv")
df["ref_clean"] = df["transcript_manual"].apply(preprocess_text)
df["hyp_clean"] = df["transcript_wav2vec2"].apply(preprocess_text)

In [ ]:
# Evaluasi Akhir
df["WER_final"] = df.apply(lambda r: wer(r["ref_clean"], r["hyp_clean"]), axis=1)
df["CER_final"] = df.apply(lambda r: cer(r["ref_clean"], r["hyp_clean"]), axis=1)

# Global Metrics
global_wer = wer(" ".join(df.ref_clean), " ".join(df.hyp_clean))
global_cer = cer(" ".join(df.ref_clean), " ".join(df.hyp_clean))

In [ ]:
# Output Evaluasi per Audio
print("\nHasil per-audio:")
print(df[["index", "WER_final", "CER_final"]].rename(columns={"WER_final": "WER", "CER_final": "CER"}))


Hasil per-audio:
   index       WER       CER
0      1  0.626866  0.317523
1      2  0.636364  0.313175
2      3  0.721519  0.372352
3      4  0.500000  0.256410
4      5  0.617225  0.322191


In [ ]:
print("Global WER:", global_wer)
print("Global CER:", global_cer)
print("Akurasi STT (WER):", 1 - global_wer)
print("Akurasi STT (CER):", 1 - global_cer)

Global WER: 0.6123399301513388
Global CER: 0.3142493638676845
Akurasi STT (WER): 0.3876600698486612
Akurasi STT (CER): 0.6857506361323156


In [ ]:
# Simpan hasil
df.to_csv("final_transcript_metrics_wav2vec2.csv", index=False)

# Perbandingan Whisper dan WAV2VEC2

In [ ]:
df_whisper = pd.read_csv("/content/final_transcript_metrics.csv")            # punya WER whisper
df_w2v2    = pd.read_csv("/content/final_transcript_metrics_wav2vec2.csv")   # punya WER wav2vec2

# Merge berdasarkan index
df_compare = df_whisper[["index", "WER_final", "CER_final"]].merge(
    df_w2v2[["index", "WER_final", "CER_final"]],
    on="index",
    suffixes=("_whisper", "_wav2vec2"))

print("Perbandingan WER & CER Whisper dan WAV2VEC2")
df_compare


Perbandingan WER & CER Whisper dan WAV2VEC2


,index,WER_final_whisper,CER_final_whisper,WER_final_wav2vec2,CER_final_wav2vec2
0,1,0.089552,0.067194,0.626866,0.317523
1,2,0.084416,0.039957,0.636364,0.313175
2,3,0.113924,0.079153,0.721519,0.372352
3,4,0.083333,0.063628,0.500000,0.256410
4,5,0.091346,0.082166,0.617225,0.322191


Pada tahap evaluasi model Speech-to-Text (STT), dilakukan perbandingan antara OpenAI Whisper dan Facebook Wav2Vec2 menggunakan dua metrik utama:
* **WER (Word Error Rate)** — mengukur kesalahan pada tingkat kata

* **CER (Character Error Rate)** — mengukur kesalahan pada tingkat karakter

Semakin rendah nilai WER/CER, semakin akurat model tersebut dalam mentranskripsi audio.

Dataset yang digunakan untuk evaluasi terdiri dari 5 video dengan durasi rata-rata ±2 menit (total ±10 menit audio).

**ANALISIS**

Dari ke-5 data, Whisper mengungguli WAV2VEC2 pada seluruh data. Hal ini menunjukkan bahwa Whisper mampu menghasilkan transkripsi yang lebih mendekati ground truth pada seluruh sampel.

Nilai error yang tinggi pada Wav2Vec2 (bahkan lebih dari 5–8 kali lipat Whisper) menunjukkan bahwa model ini:
1. Kurang stabil untuk interview
2. Lebih sensitif terhadap intonasi dan jeda
3. Berpotensi salah mengenali kata, membuang kata, atau menambahkan kata yang tidak diucapkan

**KESIMPULAN**
Berdasarkan metrik WER & CER, dapat disimpulkan bahwa Whisper dipilih sebagai model STT untuk sistem AI Interview Assessment

# STT + Groq API Untuk Penilaian Otomatis Based On Rubrics

In [ ]:
# Load JSON Rubrics Interview
# Download rubrics_interview.json dari Google Drive
!gdown --id 1METGlHn1m5ZOuSrNKL8NVsvIn6zKLSF9 -O rubrics_interview.json




/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1METGlHn1m5ZOuSrNKL8NVsvIn6zKLSF9
To: /content/rubrics_interview.json
100% 6.55k/6.55k [00:00<00:00, 22.4MB/s]


In [ ]:
# Load transcript hasil Whisper
df = pd.read_csv("final_transcript_metrics.csv")

# Load rubrik penilaian
with open("rubrics_interview.json") as f:
    rubrics = json.load(f)

In [ ]:
pprint.pprint(rubrics['1'])

{'question': 'Can you share any specific challenges you faced while working on '
             'certification and how you overcame them?',
 'rubric': {'0': ['Unanswered'],
            '1': ['Minimal or Vague Response'],
            '2': ['General Challenge Mentioned without Details'],
            '3': ['Describes at least one specific challenge related to '
                  'building machine learning models with TensorFlow.',
                  'Provides a basic explanation of how the challenge was '
                  'overcome.',
                  'Explanation may be brief or lack depth.',
                  'Shows some understanding but lacks detailed insight.'],
            '4': ['Provides a detailed description of specific challenges '
                  'encountered during the certification.',
                  'Offers clear explanations of how each challenge was '
                  'overcome.',
                  'Demonstrates strong understanding of technical aspects and '
         

In [ ]:
# Initialize Groq client
client = Groq(api_key="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") #masukkan atau ganti api key

print("Groq client initialized!")

Groq client initialized!


In [ ]:
def make_prompt(question: str, answer: str, rubric: dict) -> str:
    # Format rubrik menjadi teks terstruktur
    rubric_text = ""
    for score in sorted(rubric.keys(), key=int):
        descriptions = rubric[score]
        rubric_text += f"\nScore {score}:\n"
        for desc in descriptions:
            rubric_text += f"- {desc}\n"

    prompt = f"""
You are an evaluator. Assess the following candidate response to the interview question.

QUESTION:
{question}

ANSWER:
{answer}

SCORING RUBRIC:
{rubric_text}

Please evaluate the answer based on the rubric and respond ONLY in JSON format:
{{
  "score": (a number from 0 to 4),
  "reason": (a short explanation why the score was given)
}}
""".strip()

    return prompt

In [ ]:
def llm_evaluate(prompt: str) -> str:
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful interview evaluator. Always respond in valid JSON format."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            model="llama-3.3-70b-versatile",
            temperature=0,
            max_tokens=300,
        )

        return chat_completion.choices[0].message.content

    except Exception as e:
        return json.dumps({"score": None, "reason": f"Error: {str(e)}"})

In [ ]:
results = []

for i, row in df.iterrows():
    idx = str(row["index"])
    question = rubrics[idx]["question"]
    rubric = rubrics[idx]["rubric"]
    answer = row["hyp_no_filler"]

    print(f"\nMenilai video ke-{idx}...")

    # Generate prompt
    prompt = make_prompt(question, answer, rubric)

    try:
        # Mendapatkan respons dari Groq
        output = llm_evaluate(prompt)
        print("Output LLM:", output[:300])

        # Mencari JSON object di dalam response
        if "{" in output:
            json_str = output[output.find("{"):output.rfind("}")+1]
            parsed = json.loads(json_str)
            score = parsed.get("score", None)
            reason = parsed.get("reason", "")
        else:
            score = None
            reason = "No valid JSON found in response"

    except Exception as e:
        score = None
        reason = f"Gagal parsing: {e}"
        print(f"Error: {e}")

    results.append({
        "index": row["index"],
        "score": score,
        "reason": reason
    })

    time.sleep(0.5)

print("\nEvaluasi selesai!")


Menilai video ke-1...
Output LLM: {
  "score": 3,
  "reason": "The candidate describes a specific challenge related to meeting the accuracy or validation loss for the evaluation matrix and provides a basic explanation of how it was overcome, such as trying different architectures and applying dropout layers, but the explanation lack

Menilai video ke-2...
Output LLM: {
  "score": 2,
  "reason": "The candidate mentions transfer learning and TensorFlow, provides some examples of pre-trained models like VGG16 and VGG19, but lacks specific details about personal experience and does not clearly explain how transfer learning benefited their projects."
}

Menilai video ke-3...
Output LLM: {
  "score": 3,
  "reason": "The candidate describes a specific TensorFlow model they built for celiac disease prediction, mentions using Keras, dense layers, dropout layers, and callback functions, and explains addressing dataset imbalance with SMOTE and synthetic oversampling. However, the explana

Menila

In [ ]:
# Format ke payload JSON
interview_scores = []

for item in results:
    interview_scores.append({
        "id": int(item["index"]),
        "score": item["score"],
        "reason": item["reason"]
    })

# Hitung rata-rata score (skip yang None)
valid_scores = [x["score"] for x in interview_scores if x["score"] is not None]
avg_score = sum(valid_scores) / len(valid_scores) if valid_scores else 0

project_score = 100
interview_normalized = (avg_score / 4) * 100
total_score = (project_score + interview_normalized) / 2

payload = {
    "assessorProfile": {
        "id": 0,
        "name": "AI Assessor (Groq Mixtral-8x7B)",
        "photoUrl": ""
    },
    "decision": "Pending",
    "reviewedAt": time.strftime("%Y-%m-%d %H:%M:%S"),
    "scoresOverview": {
        "project": round(project_score, 2),
        "interview": round(avg_score, 2),
        "interview_normalized": round(interview_normalized, 2),
        "total": round(total_score, 2)
    },
    "reviewChecklistResult": {
        "project": [],
        "interviews": {
            "minScore": 0,
            "maxScore": 4,
            "scores": interview_scores
        }
    },
    "overallNotes": "Generated automatically by Groq API based on rubric scoring."
}

# Simpan ke file JSON
with open("interview_scores_groq.json", "w") as f:
    json.dump(payload, f, indent=2)

print("\nPayload penilaian disimpan ke interview_scores_groq.json")
print(f"Average Interview Score: {avg_score:.2f}/4")
print(f"Interview Normalized: {interview_normalized:.2f}/100")
print(f"Total Score: {total_score:.2f}/100")


Payload penilaian disimpan ke interview_scores_groq.json
Average Interview Score: 2.60/4
Interview Normalized: 65.00/100
Total Score: 82.50/100


# MERGE JSON OUTPUT

In [ ]:
import json

# File JSON kamu
transcript_file = "transcripts.json"
score_file = "interview_scores_groq.json"

# Baca transcripts.json
with open(transcript_file, "r") as f:
    transcripts = json.load(f)

# Baca interview_scores_groq.json
with open(score_file, "r") as f:
    interview_scores = json.load(f)

# Gabungkan dalam satu struktur yang rapi
merged = {
    "transcripts": transcripts,
    "interview_scores": interview_scores
}

# Simpan file gabungan
with open("final_payload.json", "w") as f:
    json.dump(merged, f, indent=2)

print("final_payload.json berhasil dibuat!")


final_payload.json berhasil dibuat!


In [ ]:
!pip freeze

absl-py==1.4.0
accelerate==1.12.0
access==1.1.9
affine==2.4.0
aiofiles==24.1.0
aiohappyeyeballs==2.6.1
aiohttp==3.13.2
aiosignal==1.4.0
aiosqlite==0.21.0
alabaster==1.0.0
albucore==0.0.24
albumentations==2.0.8
ale-py==0.11.2
alembic==1.17.2
altair==5.5.0
annotated-types==0.7.0
antlr4-python3-runtime==4.9.3
anyio==4.12.0
anywidget==0.9.21
argon2-cffi==25.1.0
argon2-cffi-bindings==25.1.0
array_record==0.8.3
arrow==1.4.0
arviz==0.22.0
astropy==7.2.0
astropy-iers-data==0.2025.12.8.0.38.44
astunparse==1.6.3
atpublic==5.1
attrs==25.4.0
audioread==3.1.0
Authlib==1.6.5
autograd==1.8.0
babel==2.17.0
backcall==0.2.0
beartype==0.22.8
beautifulsoup4==4.13.5
betterproto==2.0.0b6
bigframes==2.30.0
bigquery-magics==0.10.3
bleach==6.3.0
blinker==1.9.0
blis==1.3.3
blobfile==3.1.0
blosc2==3.12.2
bokeh==3.7.3
Bottleneck==1.4.2
bqplot==0.12.45
branca==0.8.2
brotli==1.2.0
CacheControl==0.14.4
cachetools==6.2.2
catalogue==2.0.10
certifi==2025.11.12
cffi==2.0.0
chardet==5.2.0
charset-normalizer==3.4.4
chex==